In [ ]:
!pip install keras-squeezenet
import cv2
import os
import numpy as np
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout, GlobalAveragePooling2D, Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
import cv2
import os
import numpy as np
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout, GlobalAveragePooling2D, Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
zip_file_path = '/content/drive/MyDrive/archive.zip'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def filter_best_images(data_path, max_images_per_class=1500):
    """Selects the best 1500 images per class, resizes them to (64, 64), and returns NumPy arrays."""
    dataset = []
    for directory in os.listdir(data_path):
        path = os.path.join(data_path, directory)
        if not os.path.isdir(path):
            continue
        count = 0  # Keep track of images per class
        for item in os.listdir(path):
            if count >= max_images_per_class:
                break
            img = cv2.imread(os.path.join(path, item))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (64, 64))  # Resize to (64, 64)
            dataset.append([img, directory])
            count += 1
    data, labels = zip(*dataset)
    return np.array(data), labels

In [ ]:
import zipfile
extracted_folder_path = '/content/extracted_contents'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [ ]:
DATA_PATH = "/content/extracted_contents/asl_alphabet_train/asl_alphabet_train"
print(os.listdir(DATA_PATH))

['T', 'P', 'B', 'nothing', 'K', 'W', 'S', 'H', 'F', 'U', 'N', 'D', 'J', 'X', 'R', 'M', 'del', 'E', 'A', 'Z', 'Q', 'L', 'O', 'space', 'V', 'G', 'I', 'Y', 'C']


In [ ]:
CODES = {"nothing": 0}
def make_labels():
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    for i in range(1, 27):
        CODES[alpha[i - 1]] = i

    CODES["del"] = 27
    CODES["space"] = 28
    return CODES

In [ ]:
def code_conv(label, codes):
    return codes[label]

In [ ]:
def load_images(data_path, max_images_per_class=1500, image_size=(64, 64)):
    """Load images, filter the best 1500 per class, resize to given size, and return NumPy arrays."""
    dataset = []
    for directory in os.listdir(data_path):
        path = os.path.join(data_path, directory)
        if not os.path.isdir(path):
            continue
        count = 0
        for item in os.listdir(path):
            if count >= max_images_per_class:
                break
            img = cv2.imread(os.path.join(path, item))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            dataset.append([img, directory])
            count += 1
    data, labels = zip(*dataset)
    return np.array(data), labels

In [ ]:
def preprocess_labels(labels, codes):
    labels = list(map(lambda x: code_conv(x, codes), labels))
    return to_categorical(labels, num_classes=len(codes))

In [ ]:
def normalize_data(data):
    return data.astype('float32') / 255.0

In [ ]:
extracted_folder_path = '/content/extracted_contents'
DATA_PATH = os.path.join(extracted_folder_path, 'asl_alphabet_train/asl_alphabet_train')

data, labels = filter_best_images(DATA_PATH)  # Filter images
codes = make_labels()
labels = preprocess_labels(labels, codes)
data = normalize_data(data)

# Data Split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)  # 30% for holdout
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15/0.7, random_state=42)  # Of 70%

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="mobilenet_model_kd.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Assume your test data and labels are in X_test, y_test (from your original code)

correct_count = 0
total_count = len(X_test)

for i in range(total_count):
  input_data = np.expand_dims(X_test[i], axis=0)  # Adjust if your model expects a different shape
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  output_data = interpreter.get_tensor(output_details[0]['index'])
  predicted_label = np.argmax(output_data[0])
  true_label = np.argmax(y_test[i])

  if predicted_label == true_label:
    correct_count += 1

accuracy = correct_count / total_count * 100

print("Accuracy of the quantized model:", accuracy, "%")


Accuracy of the quantized model: 91.77777777777779 %


In [ ]:
from google.colab import files

model.save('mobilenet_model_kd.h5')

# Download the model file
files.download('mobilenet_model_kd.h5')

In [ ]:
import matplotlib.pyplot as plt

# Extract training and validation accuracy and loss from history
training_accuracy = history_student.history['accuracy']
validation_accuracy = history_student.history['val_accuracy']
loss = history_student.history['loss']
val_loss = history_student.history['val_loss']

In [ ]:
# Plot accuracy
plt.figure(figsize=(10, 5))
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot loss
plt.figure(figsize=(10, 5))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()